Step 1 - Import necassary modules

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.cross_validation import LeaveOneOut, cross_val_predict
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from chris_rasmus_graph_features import *
from count_lagged_complaints import *
from processing import *

Step 2 - Load data

In [2]:
offc = pd.read_csv('/Users/Mehraveh/Desktop/policy_lab/toy.officer_data.csv',sep=',')
cmpl = pd.read_csv('/Users/Mehraveh/Desktop/policy_lab/toy.complaint_data.csv', sep=',')

Step 3 - Feature extraaction/insertion

In [ ]:
# Adds the number of complaints done by each officer in previous years (lags)
last_train_year = 2012
number_of_years_to_go_back = 4
r = add_complaints_by_year(last_train_year, number_of_years_to_go_back, cmpl, offc) 

# Making X vector (features)
X = r

#Dropping "FirstName", "LastName", "BirthYear","Rank"
X = X.drop(X.columns[[0,1,5,7,11]], axis=1)
# Getting years from apponted dates and normalizing
# temp = X[X.columns[0]].astype(str).str[0:4].astype(int)
# X[X.columns[0]] = (temp-np.mean(temp))/np.std(temp)
X[X.columns[0]] = X[X.columns[0]].astype(str).str[0:4].astype(int)


#Encoding categorical variables
X_cat = X[X.columns[[1,2]]]
XX=X_cat.apply(LabelEncoder().fit_transform)
X[X.columns[[1,2]]] = XX
np.where(np.isnan(X))
X_race = pd.get_dummies(X['race'])
X_race.columns = ['race0','race1','race2','race3']
X = X.drop(X.columns[[1]], axis=1)
X = X.join(X_race, how='outer')




Y = add_complaints_by_year(2015, 3, cmpl, offc)

Y=Y[[Y.columns[Y.shape[1]-1],Y.columns[Y.shape[1]-2],Y.columns[Y.shape[1]-3]]]
Y=Y.sum(axis=1)


In [3]:
#Adding graph features

scale_days = 365
base_year = 2012
last_train_year = 2012
base_month = 12
base_day = 31
complaint_df = add_lag_to_complaints(cmpl, scale_days, base_year, base_month, base_day)

# build bipartite graph
G = build_bipartite_graph(complaint_df)

lag = 4
officer_ids = offc['officer_id'].unique()
deg_thresh = 5

# get feature dictionaries
num_nbr_complaints_dict = num_of_nbr_complaints(G, officer_ids, lag)
num_high_offender_nbrs = num_high_offender_nbrs(G, officer_ids, deg_thresh)

# 

# num_nbr_complaints_dict[]

TypeError: 'method' object is not subscriptable